In [1]:
import os
os.chdir("..")
import descriptor.three_desc_model as tdm
from descriptor.cutoffs import Polynomial
import numpy as np
import pandas as pd
from scipy.optimize import minimize,fmin
import matplotlib.pyplot as plt
import GPy
from IPython.display import display
dataset = pd.read_csv('turb_data.csv')
dataset= dataset.drop(['ID','turb_num'], axis=1)
symbol="Siemens"
numsims=dataset["IDnum"].iloc[-1]+1
turb = [symbol]
nl=tdm.NeighborlistCalculator(cutoff=4001,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)

In [2]:
def f(x):
    Gs = {"Siemens": [{"type":"G2", "turbine":"Siemens","eta":x[0], "offset": x[3]},
                  {"type":"G4", "elements":"Siemens","eta":x[1], "gamma":x[4]},
                  {"type":"G6", "elements":"Siemens","eta":x[2], "gamma": x[5]}
                 ]}
    finpr=tdm.FingerprintCalculator(cutoff=4001,Gs=Gs,Rct=3000,delta_R=100,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)
    count=0
    dataset_fp=np.empty(shape=(0, 3))
    dataset_ws=np.empty(shape=(0, 1))
    dataset_rws=np.empty(shape=(0, 1))
    dataset_pos=np.empty(shape=(0, 2))
    for i in range(numsims):
        numturb=dataset["num_tot_turb"].iloc[count]
        position = np.empty((numturb,2))
        rws = np.empty((numturb,1))
        ws = np.empty((numturb,1))
        #fp = np.empty((numturb,3))
        for k in range(numturb):
            position[k,0]=dataset.at[count,"x_coord"]
            position[k,1]=dataset.at[count,"y_coord"]
            rws[k,0]=dataset["ref_wind_speed"].iloc[count]
            ws[k,0]=dataset.at[count,"wind_speed"]
            count = count+ 1
        neigh=nl.calculate(turb*numturb,position)
        fingerprints=finpr.calculate(turb*numturb,position,neigh,symbol)
        fingerprints=np.array(fingerprints)
        dataset_fp=np.append(dataset_fp,fingerprints,axis=0)
        dataset_rws=np.append(dataset_rws,rws,axis=0)
        dataset_ws=np.append(dataset_ws,ws,axis=0)
        dataset_pos=np.append(dataset_pos,position,axis=0)
    dataset_df=np.concatenate((dataset_fp, dataset_ws, dataset_rws),axis=1)
    dataset_df = pd.DataFrame(dataset_df, columns = ['Fingerprint(G2)','Fingerprint(G4)','Fingerprint(G6)','Wind_Speed','Ref_Wind_Speed'])
    #X=dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)","Wind_Speed"]].to_numpy()
    #Y=dataset[["Ref_Wind_Speed"]].to_numpy()
    train_dataset = dataset_df.sample(frac=0.8, random_state=0)
    test_dataset = dataset_df.drop(train_dataset.index)
    Xtrain=train_dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)","Wind_Speed"]].to_numpy()
    Ytrain=train_dataset[["Ref_Wind_Speed"]].to_numpy()
    Xtest=test_dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)","Wind_Speed"]].to_numpy()
    Ytest=test_dataset[["Ref_Wind_Speed"]].to_numpy()
    # define kernel
    ker = GPy.kern.RBF(4,lengthscale=0.1)# + GPy.kern.White(2)
    # create simple GP model
    m = GPy.models.GPRegression(Xtrain,Ytrain,ker)
    # optimize and plot
    m.optimize(max_f_eval = 1000)

    [Ypred,Yvar]=m.predict(Xtest)
    return np.max(Ypred-Ytest)

In [3]:
x0 = [5.3, 5., 2., 1., 8., 1.]
bounds= ((0,10),(0,10),(0,10),(0,10),(0,10),(0,10))
res = minimize(f, x0, bounds= bounds, options={'disp':True})

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.56012D-01    |proj g|=  7.26081D-02

At iterate    1    f=  8.49742D-01    |proj g|=  2.78172D-01

At iterate    2    f=  8.32250D-01    |proj g|=  9.19490D-02

At iterate    3    f=  8.03219D-01    |proj g|=  2.25593D-01

At iterate    4    f=  6.78540D-01    |proj g|=  3.24361D-01
  ys=-1.610E-01  -gs= 7.541E-02 BFGS update SKIPPED

At iterate    5    f=  6.66316D-01    |proj g|=  1.65737D-01

At iterate    6    f=  6.05231D-01    |proj g|=  3.29375D-01

At iterate    7    f=  5.51383D-01    |proj g|=  7.86155D+00

At iterate    8    f=  5.51381D-01    |proj g|=  7.86155D+00
  Positive dir derivative in projection 
  Using the backtracking step 



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate    9    f=  5.51373D-01    |proj g|=  7.86155D+00
  ys=-4.107E-05  -gs= 5.181E-05 BFGS update SKIPPED

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6     10     90     21     2     6   7.862D+00   5.514D-01
  F =  0.55137294198080244     

ABNORMAL_TERMINATION_IN_LNSRCH                              



 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.


In [4]:
res

      fun: 0.5513886036340123
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([1574.41318496, 1651.51308123, 1643.3032676 ,  348.35955861,
       1560.8296917 , -781.58480092])
  message: 'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 630
      nit: 9
     njev: 90
   status: 2
  success: False
        x: array([6.03533325, 3.35138237, 0.920186  , 1.17618377, 7.86155144,
       2.42807815])